In [1]:
import os
import sys

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import plotly.subplots as subplots

from typing import Union, Literal

sys.path.append(os.path.abspath('..'))
from graphmodex import plotlymodex

In [2]:
n_rows = 10000
n_cols = 20

dates = pd.date_range(start="2025-01-01", end="2025-05-30", periods=n_rows)

data = {"date": dates}

for i in range(3):
    data[f"cat_30_{i}"] = np.random.choice([f"{j}" for j in range(30)], size=n_rows)
for i in range(2):
    data[f"cat_5_{i}"] = np.random.choice([f"{j}" for j in ['a','b','c','d','e']], size=n_rows)

# Generate 5 integer columns (ordered, like credit history length, loan duration)
for i in range(5):
    data[f"int_{i}"] = np.random.randint(0, 100, size=n_rows)

# Generate 5 float columns (e.g. income, loan amount, utilization)
for i in range(5):
    data[f"float_{i}"] = np.random.normal(loc=5000, scale=2000, size=n_rows)
    data[f"float_{i+4}"] = np.random.normal(loc=0, scale=1, size=n_rows)

# Generate 2 columns with mostly missing values
data["mostly_nan_1"] = np.where(np.random.rand(n_rows) < 0.95, np.nan, np.random.rand(n_rows))
data["mostly_nan_2"] = np.where(np.random.rand(n_rows) < 0.9, np.nan, np.random.rand(n_rows))

# Generate 1 column with some special missing values (inf, -inf)
special_col = np.random.normal(loc=100, scale=50, size=n_rows)
special_col[np.random.choice(n_rows, size=10, replace=False)] = np.inf
special_col[np.random.choice(n_rows, size=10, replace=False)] = -np.inf
special_col[np.random.choice(n_rows, size=100, replace=False)] = np.nan
data["special_missing"] = special_col

# Fixing the target column to handle NaNs by using float type
target = np.random.choice([0, 1], size=n_rows).astype(float)
mask = np.random.rand(n_rows) < 0.2
target[mask] = np.nan
data["target"] = target

for i, col in enumerate([col for col in data if col.startswith("cat_")]):
    if (i == 3) or (i == 1): continue
    nan_indices = np.random.choice(n_rows, size=50, replace=False)
    temp_col = np.array(data[col], dtype=object)
    temp_col[nan_indices] = np.nan
    data[col] = temp_col

# Insert a few NaNs into the float columns
for i, col in enumerate([col for col in data if col.startswith("float_")]):
    if (i == 3): break
    nan_indices = np.random.choice(n_rows, size=30, replace=False)
    temp_col = data[col]
    temp_col[nan_indices] = np.nan
    data[col] = temp_col

# Convert to DataFrame
df = pd.DataFrame(data)
df['month'] = df['date'].dt.month

In [ ]:
plotlymodex.animated_scatter(
    df,
    x='float_0',
    y='float_1',
    animation_frame='month',
    color='cat_5_0',
    colorscale=plotlymodex.list_colorscale(n=5, colorscale='dense'),
    auto_fit=False
)

In [3]:
plotlymodex.animated_bar(
    df,
    x='cat_5_0',
    y='float_1',
    animation_frame='month',
    agg_func='count',
    pattern_shape='cat_5_0',
    color='cat_5_0',
    barmode='group',
    colorscale=plotlymodex.list_colorscale(n=5, colorscale='dense'),
)

In [6]:
import plotly.express as px

fig = px.bar(df.groupby(['cat_5_0','month',  'cat_30_0'])['date'].count().reset_index(), 
            x="date", y="cat_5_0", animation_frame="month",
            color="cat_30_0", 
            color_discrete_sequence=px.colors.qualitative.Vivid_r, barmode='relative', orientation='h', title='x', subtitle='X')

plotlymodex.main_layout(fig, barmode='group')

### PK

In [7]:
time_points = [0, 0.17, 0.33, 0.5, 0.75, 1, 1.5, 2, 2.5, 3, 4, 6, 8, 10, 12, 24]

# Subjects
columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

# Plasma concentrations for each subject over time
data = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [571.7, 229.4, 52.8, 233.2, 220.5, 259.9, 13.4, 18.1, 83.1, 211.1, 143.3, 95.5],
    [780.0, 243.6, 68.0, 241.4, 300.3, 358.2, 9.0, 28.1, 140.6, 329.1, 321.6, 385.4],
    [629.3, 206.9, 138.8, 218.8, 560.0, 231.9, 219.0, 40.3, 336.8, 844.9, 386.5, 323.8],
    [551.7, 153.4, 148.6, 198.9, 501.7, 184.4, 134.2, 30.1, 259.9, 668.7, 330.6, 226.2],
    [432.0, 140.0, 164.6, 169.8, 310.6, 155.7, 30.1, 168.3, 222.4, 542.4, 409.6, 197.0],
    [361.9, 152.0, 143.6, 196.7, 333.7, 148.6, 267.9, 264.7, 200.4, 456.6, 342.5, 170.0],
    [352.6, 184.9, 131.7, 232.4, 318.8, 223.5, 221.3, 247.8, 234.4, 301.6, 342.5, 180.6],
    [302.6, 248.3, 144.4, 249.6, 307.8, 207.3, 253.1, 343.9, 276.9, 336.5, 273.3, 187.4],
    [329.7, 264.9, 154.9, 228.8, 387.4, 315.8, 301.4, 328.1, 329.2, 291.2, 217.6, 209.5],
    [244.5, 209.9, 166.2, 201.3, 434.1, 220.7, 204.6, 270.8, 280.1, 216.2, 200.8, 339.0],
    [120.7, 162.2, 117.6, 147.7, 311.8, 136.9, 132.5, 152.8, 214.5, 128.1, 132.0, 182.0],
    [63.0, 137.5, 109.1, 114.8, 270.4, 93.9, 90.2, 105.0, 188.6, 102.8, 90.0, 148.2],
    [31.7, 127.4, 95.5, 5.1, 109.5, 50.1, 29.2, 76.6, 122.6, 86.0, 59.5, 50.7],
    [11.6, 97.0, 80.5, 33.4, 111.4, 47.2, 39.6, 49.6, 88.8, 46.4, 41.9, 91.1],
    [0.0, 45.7, 43.2, 1.3, 31.1, 16.3, 17.8, 31.3, 38.7, 32.0, 18.7, 17.5],
]

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)
df.insert(0, "t", time_points)

In [8]:
subjects = ['B', 'E', 'F', 'I', 'L']
num_subjects = len(subjects)

grayscale_colors = plotlymodex.list_colors(n=len(subjects), colors=['#d1d1d1', '#0f0f0f'])

fig = go.Figure()

for subject, color in zip(subjects, grayscale_colors):
    fig.add_trace(go.Scatter(
        x=df.t, y=df[f'{subject}'], name=subject,
        line=dict(color=color)
    ))

plotlymodex.main_layout(
    fig, title='Veralipride Plasma Concentration', width=900, height=450,
    x='t [h]', y='C [ng/mL]', 
)